# Setup packages

In [1]:
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
%matplotlib inline

# Loading data

In [2]:
'''
# Deprecated code
import pickle   # data
import sys      # directories
sys.path.append("code") # contains CNN model
from gazelle_utils import *
'''

'\n# Deprecated code\nimport pickle   # data\nimport sys      # directories\nsys.path.append("code") # contains CNN model\nfrom gazelle_utils import *\n'

In [12]:
# Is this Owen's instance or Matthew's?
#### CHANGE THE VARIABLE RIGHT HERE ####
instance = 'matthew'   # 'matthew' or 'owen'

In [17]:
from os import listdir
from os.path import isfile, join
if (instance == 'matthew'):
    datapath = "data_CNN/clean"
else:
    datapath = "../../Owen/gazelle-github-Owen/data_CNN/clean"
onlyfiles = [f for f in listdir(datapath) if isfile(join(datapath, f))]
onlyfiles.sort()
print "There are ", len(onlyfiles), " files: ", onlyfiles

There are  8  files:  ['XYArray1.npy', 'XYArray2.npy', 'XYArray3.npy', 'XYArray4.npy', 'data1.npy', 'data2.npy', 'data3.npy', 'data4.npy']


In [23]:
import re
fileNums = []
for f in onlyfiles:
    fileNums.append(int(re.sub("[^0-9]", "", f)))
fileNums = list(set(fileNums))
print fileNums

[1, 2, 3, 4]


In [16]:
# Debug
if len(onlyfiles) % 3 is not 0: 
    print "There is an odd number of files. Look back at data folder."
else:
    print "Good to go."

There is an odd number of files. Look back at data folder.


In [6]:
import os
def load_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        return np.load(f)

def load_all(ROOT, fileno):
    """ load all of cifar """
    xs = []
    ys = []
    f = os.path.join(ROOT, 'data%d.npy' % (fileno, ))
    g = os.path.join(ROOT, 'XYArray%d.npy' % (fileno, ))
    X = load_batch(f)
    Y = load_batch(g)
    xs.append(X)
    ys.append(Y)    
    Xall = np.concatenate(xs)
    yall = np.concatenate(ys)
    print "Xall shape: ", Xall.shape
    print "yall shape: ", yall.shape
    del X, Y
    return Xall, yall


def get_and_segment_data(fileno, subtract_mean=False, directory = "data_CNN/clean", train_cut = 0.7,
                    val_cut = 0.2, test_cut = 0.1):
    """
    Load the dataset from disk and maybe perform preprocessing to prepare
    it for classifiers. These are the same steps as we used for the SVM, but
    condensed to a single function.
    """
    # Load the raw data
    X_all, y_all = load_all(directory, fileno)
    
    # Slicing the data
    num_training = int(X_all.shape[0] * train_cut)
    num_validation = int(X_all.shape[0] * val_cut)
    num_testing = int(X_all.shape[0] * test_cut)

    # Subsample the data
    mask = list(range(num_training))
    X_train = X_all[mask]
    y_train = y_all[mask]
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_all[mask]
    y_val = y_all[mask]
    mask = list(range(num_training + num_validation, num_training + num_validation + num_testing))
    X_test = X_all[mask]
    y_test = y_all[mask]

    # Normalize the data: subtract the mean image
    if subtract_mean:
        mean_image = np.mean(X_train, axis=0)
        X_train -= mean_image
        X_val -= mean_image
        X_test -= mean_image

    return X_train.astype('float32'), y_train.astype('float32'), X_val.astype('float32'), y_val.astype('float32'), X_test.astype('float32'), y_test.astype('float32')

# Model

In [7]:
# clear old variables
tf.reset_default_graph()

# setup input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 144, 144, 3, 4])
y = tf.placeholder(tf.float32, [None, 2])
is_training = tf.placeholder(tf.bool)

def cnn_model_fn(features, labels):
    # features = Tensor("Print:0", shape=(?, 144, 144, 3, 4), dtype=float32)

    # GC Input Layer
    # we have 4 inputs in the order: right eye, left eye, face, face grid (bound by dim #4 of value 4)
    #   Each one is [batch_size, width, height, channels] = num_batches x 144 x 144 x 3.

    R_eye = tf.squeeze(tf.slice(features, [0,0,0,0,0], [-1, 144, 144, 3, 1]), axis=4)
    L_eye = tf.squeeze(tf.slice(features, [0,0,0,0,1], [-1, 144, 144, 3, 1]), axis=4)
    face  = tf.squeeze(tf.slice(features, [0,0,0,0,2], [-1, 144, 144, 3, 1]), axis=4)
    fgrid = tf.squeeze(tf.slice(features, [0,0,0,0,3], [-1, 144, 144, 3, 1]), axis=4)
    # Tensor("Print_1:0", shape=(?, 1, 144, 144, 3), dtype=float32)


    # Convolutional Layer #1
    # Input Tensor Shape: [batch_size, 144, 144, 3]
    # Output Tensor Shape: [batch_size, 144, 144, 96]
    conv_ER1 = tf.layers.conv2d(
      inputs=R_eye,
      filters=96,
      kernel_size=[11,11],
      padding="same",
      activation=tf.nn.relu)
    conv_EL1 = tf.layers.conv2d(
      inputs=L_eye,
      filters=96,
      kernel_size=[11,11],
      padding="same",
      activation=tf.nn.relu)
    conv_F1  = tf.layers.conv2d(
      inputs=face,
      filters=96,
      kernel_size=[11,11],
      padding="same",
      activation=tf.nn.relu)
    # Tensor("Print:0", shape=(?, 144, 144, 96), dtype=float32)


    # Pooling Layer 1
    # Input Tensor Shape: [batch_size, 144, 144, 96]
    # Output Tensor Shape: [batch_size, 72, 72, 96]
    conv_ER1_pooled = tf.layers.max_pooling2d(
      inputs=conv_ER1,
      pool_size=[2,2],
      strides=2)
    conv_EL1_pooled = tf.layers.max_pooling2d(
      inputs=conv_EL1,
      pool_size=[2,2],
      strides=2)
    conv_F1_pooled = tf.layers.max_pooling2d(
      inputs=conv_F1,
      pool_size=[2,2],
      strides=2)


    # Convolutional Layer #2
    # Input Tensor Shape: [batch_size, 72, 72, 96]
    # Output Tensor Shape: [batch_size, 72, 72, 256]
    conv_ER2 = tf.layers.conv2d(
      inputs=conv_ER1_pooled,
      filters=256,
      kernel_size=[5,5],
      padding="same",
      activation=tf.nn.relu)
    conv_EL2 = tf.layers.conv2d(
      inputs=conv_EL1_pooled,
      filters=256,
      kernel_size=[5,5],
      padding="same",
      activation=tf.nn.relu)
    conv_F2  = tf.layers.conv2d(
      inputs=conv_F1_pooled,
      filters=256,
      kernel_size=[5,5],
      padding="same",
      activation=tf.nn.relu)

    # Convolutional Layer #3
    # Input Tensor Shape: [batch_size, 72, 72, 256]
    # Output Tensor Shape: [batch_size, 72, 72, 384]
    conv_ER3 = tf.layers.conv2d(
      inputs=conv_ER2,
      filters=384,
      kernel_size=[3,3],
      padding="same",
      activation=tf.nn.relu)
    conv_EL3 = tf.layers.conv2d(
      inputs=conv_EL2,
      filters=384,
      kernel_size=[3,3],
      padding="same",
      activation=tf.nn.relu)
    conv_F3  = tf.layers.conv2d(
      inputs=conv_F2,
      filters=384,
      kernel_size=[3,3],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer 2
    # Input Tensor Shape: [batch_size, 72, 72, 384]
    # Output Tensor Shape: [batch_size, 36, 36, 384]
    conv_ER3_pooled = tf.layers.max_pooling2d(
      inputs=conv_ER3,
      pool_size=[2,2],
      strides=2)
    conv_EL3_pooled = tf.layers.max_pooling2d(
      inputs=conv_EL3,
      pool_size=[2,2],
      strides=2)
    conv_F3_pooled  = tf.layers.max_pooling2d(
      inputs=conv_F3,
      pool_size=[2,2],
      strides=2)


    # Convolutional Layer #4
    # Input Tensor Shape: [batch_size, 36, 36, 384]
    # Output Tensor Shape: [batch_size, 36, 36, 64]
    conv_ER4 = tf.layers.conv2d(
      inputs=conv_ER3_pooled,
      filters=64,
      kernel_size=[1,1],
      padding="same",
      activation=tf.nn.relu)
    conv_EL4 = tf.layers.conv2d(
      inputs=conv_EL3_pooled,
      filters=64,
      kernel_size=[1,1],
      padding="same",
      activation=tf.nn.relu)
    conv_F4  = tf.layers.conv2d(
      inputs=conv_F3_pooled,
      filters=64,
      kernel_size=[1,1],
      padding="same",
      activation=tf.nn.relu)


    # ----------------------------------

    # Dense Layers: Eyes
    # Flatten tensors into a batch of vectors, then feed to dense layer
    # For each (of the 2 eyes):
    #   Input Tensor Shape (flatten): [batch_size, 36, 36, 64]
    #   Output Tensor Shape (flatten): [batch_size, 36 * 36 * 64]
    # Concatenate:
    #   Final Output Tensor Shape: [batch_size, 36 * 36 * 64 * 2]

    # Dense Layer Eyes: 128 units
    ER_flat = tf.reshape(conv_ER4, [-1, 36 * 36 * 64])
    EL_flat = tf.reshape(conv_EL4, [-1, 36 * 36 * 64])
    eye_flat = tf.concat([ER_flat, EL_flat], axis=1)
    dense_eyes = tf.layers.dense(inputs=eye_flat, units=128, activation=tf.nn.relu)

    # Dense Layers: Face. 128, 64
    F_flat = tf.reshape(conv_F4, [-1, 36 * 36 * 64])
    # Dense Layer Face 1: 128 units
    dense_face1 = tf.layers.dense(inputs=F_flat, units=128, activation=tf.nn.relu)
    # Dense Layer Face 2: 64 units
    dense_face2 = tf.layers.dense(inputs=dense_face1, units=64, activation=tf.nn.relu)

    # Dense Layers: Face Grid boolean mask
    #   Post-pooling Tensor SHape: [batch_size, 72, 72]
    fgrid_mask = tf.slice(fgrid, [0,0,0,0], [-1, 144, 144, 1])
    fgrid_pooled = tf.layers.max_pooling2d(
      inputs=fgrid_mask,
      pool_size=[2,2],
      strides=2)
    fgrid_flat = tf.reshape(fgrid_pooled, [-1, 72 * 72])
    # Dense Layer Face-grid 1: 256 units
    dense_fgrid1 = tf.layers.dense(inputs=fgrid_flat, units=128, activation=tf.nn.relu)
    # Dense Layer Face-grid 1: 128 units
    dense_fgrid2 = tf.layers.dense(inputs=dense_fgrid1, units=64, activation=tf.nn.relu)


    # Final Dense Layers
    #   concatenate tensors: eyes, face, face-grid.
    combined_flat = tf.concat([dense_eyes, dense_face2, dense_fgrid2], axis=1) # shape [batch_size, 128 + 64 + 64]
    dense_final = tf.layers.dense(inputs=combined_flat, units=128, activation=tf.nn.relu)
    xy_output = tf.layers.dense(inputs=dense_final, units=2)
    return xy_output

In [ ]:
xy_output = cnn_model_fn(X,y)
error = tf.sqrt(tf.losses.mean_squared_error(labels=y, predictions=xy_output))

# define our optimizer
optimizer = tf.train.AdamOptimizer(1e-5) # select optimizer and set learning rate
train_step = optimizer.minimize(error)

In [ ]:
'''
def show_images(images):
    images = np.reshape(images, [images.shape[0], -1])  # images reshape to (batch_size, D)
    sqrtn = int(np.ceil(np.sqrt(images.shape[0])))
    sqrtimg = int(np.ceil(np.sqrt(images.shape[1])))

    fig = plt.figure(figsize=(sqrtn, sqrtn))
    gs = gridspec.GridSpec(sqrtn, sqrtn)
    gs.update(wspace=0.05, hspace=0.05)

    for i, img in enumerate(images):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(img.reshape([sqrtimg,sqrtimg]))
    return
'''

def run_model(session, predict, loss_val, files, num_batches,
              epochs=1, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    #correct_prediction = tf.equal(tf.argmax(predict,1), y)
    #error = tf.sqrt(tf.losses.mean_squared_error(labels=labels, predictions=xy_output))
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    # LATER train_indicies = np.arange(Xd.shape[0])
    # LATER np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [loss_val]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    print "num_batches: ", num_batches
    for e in range(epochs):
        # keep track of losses and accuracy
        numImages = 0
        errors = []
        # make sure we iterate over the dataset once
        for i in range(num_batches):
            # generate indicies for the batch
            #start_idx = (i*batch_size)%X_train.shape[0]
            #idx = train_indicies[start_idx:start_idx+batch_size]
            X_train, y_train, X_val, y_val, _, _ = get_and_segment_data(i+1)
            print "We got to here"
            print "X_train: ", X_train.shape
            
            # create a feed dictionary for this batch
            feed_dict = {X: X_train,
                         y: y_train,
                         is_training: training_now }
            # get batch size
            actual_batch_size = y_train.shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            error, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            errors.append(error*actual_batch_size)
            numImages += actual_batch_size
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training error = {1:.3g} and average error of {2:.2g}"\
                      .format(iter_cnt,error,np.sum(errors)/actual_batch_size))
        print "We have processed ", numImages, " images."
        total_loss = np.sum(errors)/numImages
        print("Epoch {1}, Overall loss = {0:.3g}".format(total_loss,e+1))
        if plot_losses:
            plt.plot(errors)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss

with tf.Session() as sess:
    with tf.device("/cpu:0"): #"/cpu:0" or "/gpu:0" 
        sess.run(tf.global_variables_initializer())
        print('Training')
        run_model(sess,xy_output,error,onlyfiles,len(onlyfiles)/2,10,100,train_step,True)
        #print('Validation')
        #run_model(sess,y_out,mean_loss,X_val,y_val,1,64)

Training
num_batches:  4
Xall shape:  (1832, 144, 144, 3, 4)
yall shape:  (1832, 2)
We got to here

In [ ]:
with get_session() as sess:
    sess.run(tf.global_variables_initializer())
    run_model(sess,G_train_step,G_loss,D_train_step,D_loss,G_extra_step,D_extra_step)

In [64]:
num_epochs = 100
files = tf.train.match_filenames_once("./data_CNN/data*.npy")
print files
FIFOdata = tf.train.string_input_producer(
    tf.train.match_filenames_once("./data_CNN/data*.npy"), num_epochs=num_epochs)
FIFOlabels = tf.train.string_input_producer(
    tf.train.match_filenames_once("./data_CNN/XYArray*.npy"), num_epochs=num_epochs)
print FIFOdata.names

<tf.Variable 'matching_filenames_23:0' shape=<unknown> dtype=string_ref>
None


In [73]:
files = tf.train.match_filenames_once("data_CNN/data*.npy")
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    print (sess.run(files))

FailedPreconditionError: Attempting to use uninitialized value matching_filenames_28
	 [[Node: _send_matching_filenames_28_0 = _Send[T=DT_STRING, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=4826793861428859587, tensor_name="matching_filenames_28:0", _device="/job:localhost/replica:0/task:0/cpu:0"](matching_filenames_28)]]

# Create model

In [51]:
sys.path.append("CNN_code")
from gazecapture_cnn import cnn_model_fn

# Create the Estimator
gazelle_estimator = learn.Estimator(
      model_fn=cnn_model_fn, model_dir="tmp/gazelle_conv_model")
# Set up logging for when the CNN trains
tensors_to_log = { "loss": "loss_tensor",
                 "x diff": "xdiff_tensor",
                 "y diff": "ydiff_tensor" } 
# Need to DEBUG: check that x diff and y diff are logging right.

logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log,
  every_n_iter=2)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4295aec6d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4295aec6d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


# Training the model

In [46]:
numFiles = len(onlyfiles) / 2 
fileno = 1
X_train, y_train, X_val, y_val, X_test, y_test = get_and_segment_data(fileno)
print('Train data shape: ', X_train.shape, X_train.dtype)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape, X_val.dtype)
print('Validation labels shape: ', y_val.shape, y_val.dtype)
print('Test data shape: ', X_test.shape, X_test.dtype)
print('Test labels shape: ', y_test.shape, y_test.dtype)

Xall shape:  (1832, 144, 144, 3, 4)
yall shape:  (1832, 2)
('Train data shape: ', (1282, 144, 144, 3, 4), dtype('float32'))
('Train labels shape: ', (1282, 2), dtype('float32'))
('Validation data shape: ', (366, 144, 144, 3, 4), dtype('float32'))
('Validation labels shape: ', (366, 2), dtype('float32'))
('Test data shape: ', (183, 144, 144, 3, 4), dtype('float32'))
('Test labels shape: ', (183, 2), dtype('float32'))


In [43]:
CNN_DATA_ROOT = "toy_CNN_data/"
train_data_file = open(CNN_DATA_ROOT + 'train_data_tiny.pkl', 'rb') #batchA size N=364. shape [N, 144,144,3,4]
train_labels_file = open(CNN_DATA_ROOT + 'train_labels_tiny.pkl', 'rb')
eval_data_file = open(CNN_DATA_ROOT + 'eval_data_tiny.pkl', 'rb') # WRONG testing data, this is in pixels. Trained on cm.
eval_labels_file = open(CNN_DATA_ROOT + 'eval_labels_tiny.pkl', 'rb')

train_data = pickle.load(train_data_file).astype('float32') #numpy arrays
train_labels = pickle.load(train_labels_file).astype('float32')
eval_data = pickle.load(eval_data_file).astype('float32')
eval_labels = pickle.load(eval_labels_file).astype('float32')

print('Train data shape: ', train_data.shape, train_data.dtype)
print('Train labels shape: ', train_labels.shape, train_labels.dtype)
print('Validation data shape: ', eval_data.shape, eval_data.dtype)
print('Validation labels shape: ', eval_labels.shape, eval_labels.dtype)


('Train data shape: ', (24, 144, 144, 3, 4), dtype('float32'))
('Train labels shape: ', (24, 2), dtype('float32'))
('Validation data shape: ', (3, 144, 144, 3, 4), dtype('float32'))
('Validation labels shape: ', (3, 2), dtype('float32'))


In [53]:
# Train the model
gazelle_estimator.fit(
  x=X_train,
  y=y_train,
  batch_size=4,
  steps=100, # At every step, does it randomly pull out 4 samples from the 364? Can test this tomorrow
  monitors=[logging_hook])

# Make our own GC accuracy metric
# Configure the accuracy metric for evaluation
metrics = {
  "Gazelle prediction mean abs. error":
      learn.MetricSpec(
          metric_fn=tf.metrics.mean_absolute_error, prediction_key="coords delta")
}

# Evaluate the model and print results
eval_results = gazelle_estimator.evaluate(
  x=eval_data, y=eval_labels, metrics=metrics)
print(eval_results)

# Very jank way to touch a file, so that if we come back to instance-1 and check the dir,
#   we can tell the CNN finished.
f = open("we-are-finished.txt", 'w')
localtime = time.localtime(time.time())
f.write(time.asctime(localtime) + '\n')
f.close()

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


...

INFO:tensorflow:Saving checkpoints for 208 into tmp/gazelle_conv_model/model.ckpt.


INFO:tensorflow:Saving checkpoints for 208 into tmp/gazelle_conv_model/model.ckpt.


INFO:tensorflow:Loss for final step: 113.756.


INFO:tensorflow:Loss for final step: 113.756.


Estimator(params=None)

In [ ]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.argmax(predict,1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%X_train.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[i:i+batch_size].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct


In [ ]:


with tf.Session() as sess:
    with tf.device("/cpu:0"): #"/cpu:0" or "/gpu:0" 
        sess.run(tf.global_variables_initializer())
        print('Training')
        run_model(sess,y_out,mean_loss,X_train,y_train,1,64,100,train_step,True)


### We will create a simple model to test our understanding of inputs


In [14]:
# clear old variables
tf.reset_default_graph()

# setup input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 144, 144, 3, 4])
y = tf.placeholder(tf.int64, [None, 2])
is_training = tf.placeholder(tf.bool)

def faceonly_model(X,labels):
    # define our weights (e.g. init_two_layer_convnet)
    features = tf.cast(X, tf.float32)
    # GC Input Layer
    # we have 4 inputs in the order: right eye, left eye, face, face grid (bound by dim #4 of value 4)
    #   Each one is [batch_size, width, height, channels] = num_batches x 144 x 144 x 3.

    R_eye = tf.squeeze(tf.slice(features, [0,0,0,0,0], [-1, 144, 144, 3, 1]), axis=4)
    L_eye = tf.squeeze(tf.slice(features, [0,0,0,0,1], [-1, 144, 144, 3, 1]), axis=4)
    face  = tf.squeeze(tf.slice(features, [0,0,0,0,2], [-1, 144, 144, 3, 1]), axis=4)
    fgrid = tf.squeeze(tf.slice(features, [0,0,0,0,3], [-1, 144, 144, 3, 1]), axis=4)
    
    conv_F1  = tf.layers.conv2d(
      inputs=face,
      filters=96,
      kernel_size=[11,11],
      padding="same",
      activation=tf.nn.relu)
    
    conv_F1_pooled = tf.layers.max_pooling2d(
      inputs=conv_F1,
      pool_size=[2,2],
      strides=2)
    
    conv_F2  = tf.layers.conv2d(
      inputs=conv_F1_pooled,
      filters=256,
      kernel_size=[5,5],
      padding="same",
      activation=tf.nn.relu)
    
    conv_F3  = tf.layers.conv2d(
      inputs=conv_F2,
      filters=384,
      kernel_size=[3,3],
      padding="same",
      activation=tf.nn.relu)
    
    conv_F3_pooled  = tf.layers.max_pooling2d(
      inputs=conv_F3,
      pool_size=[2,2],
      strides=2)

    conv_F4  = tf.layers.conv2d(
      inputs=conv_F3_pooled,
      filters=64,
      kernel_size=[1,1],
      padding="same",
      activation=tf.nn.relu)
    
    # Dense Layers: Face. 128, 64
    F_flat = tf.reshape(conv_F4, [-1, 36 * 36 * 64])
    # Dense Layer Face 1: 128 units
    dense_face1 = tf.layers.dense(inputs=F_flat, units=128, activation=tf.nn.relu)
    # Dense Layer Face 2: 64 units
    dense_final = tf.layers.dense(inputs=dense_face1, units=64, activation=tf.nn.relu)
    
    xy_output = tf.layers.dense(inputs=dense_final, units=2)
    
    return xy_output

xy_output = faceonly_model(X,y)

# define our loss
mean_loss = tf.losses.mean_squared_error(labels=y, predictions=xy_output)

# define our optimizer
optimizer = tf.train.AdamOptimizer(5e-4) # select optimizer and set learning rate
train_step = optimizer.minimize(mean_loss)

In [18]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False):
    # have tensorflow compute accuracy
    correct_prediction = tf.equal(tf.argmax(predict,1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [mean_loss,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%X_train.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }
            # get batch size
            actual_batch_size = yd[i:i+batch_size].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct

X_train = train_data
y_train = train_labels
with tf.Session() as sess:
    with tf.device("/cpu:0"): #"/cpu:0" or "/gpu:0" 
        sess.run(tf.global_variables_initializer())
        print('Training')
        run_model(sess,xy_output,mean_loss,X_train,y_train,1,3,3,train_step,True)

Training


InvalidArgumentError: Incompatible shapes: [3] vs. [3,2]
	 [[Node: Equal_1 = Equal[T=DT_INT64, _device="/job:localhost/replica:0/task:0/cpu:0"](ArgMax_1, _recv_Placeholder_1_0)]]

Caused by op u'Equal_1', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/mdkim/assignment1/.env/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/mdkim/assignment1/.env/local/lib/python2.7/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/home/mdkim/assignment1/.env/local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 405, in start
    ioloop.IOLoop.instance().start()
  File "/home/mdkim/assignment1/.env/local/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self

# The below is discontinued code

In [4]:
# We are testing the Gazelle CNN on the *tiny* dataset right now:
# train_data_tiny, train_labels_tiny, eval_data_tiny, eval_labels_tiny
# Load training and eval data from GazeCapture dataset
'''
Discontinued
'''
CNN_DATA_ROOT = "toy_CNN_data/"
train_data = pickle.load(open(CNN_DATA_ROOT + 'train_data_tiny.pkl', 'rb'))
train_labels = pickle.load(open(CNN_DATA_ROOT + 'train_labels_tiny.pkl', 'rb'))
eval_data = pickle.load(open(CNN_DATA_ROOT + 'eval_data_tiny.pkl', 'rb'))
eval_labels = pickle.load(open(CNN_DATA_ROOT + 'eval_labels_tiny.pkl', 'rb'))

print (train_data.shape)
print (train_labels.shape)
print (eval_data.shape)
print (eval_labels.shape)

(24, 144, 144, 3, 4)
(24, 2)
(3, 144, 144, 3, 4)
(3, 2)


In [28]:
# First populate X_train and y_train with the first data chunks
firstdatapath = datapath + "/" + 'data1.npy'
firstXYpath = datapath + "/" + 'XYArray1.npy'
X_train = np.load(firstdatapath)
y_train = np.load(firstXYpath)

# Concatenate all the other data
for i in xrange(len(onlyfiles) / 2 + 1):
    if i is 0:
        continue
    Xpath = datapath + "/" + 'data' + str(i) + '.npy'
    ypath = datapath + "/" + 'XYArray' + str(i) + '.npy'
    #print "new X: ", np.load(Xpath).shape
    X_train = np.concatenate((X_train, np.load(Xpath)))
    y_train = np.concatenate((y_train, np.load(ypath)))

print "X_train: ", X_train.shape
print "y_train: ", y_train.shape

MemoryError: 